In [1]:
import numpy as np
import subprocess
import os,glob,shutil
import tarfile

## Create SLHA files from a txname template

### Template file:

In [2]:
template = './wino_template.slha'

### Lines to be replaced

In [3]:
lines = ['   1000022     5.99716797E+02   #  z1ss','   1000024     5.99890015E+02   #  w1ss',
         'DECAY   1000024  6.6e-16   # W1SS+ decays']
newlines = ['   1000022     %1.3f           # ~chi_10','   1000024     %1.3f               # ~chi_1+',
         'DECAY   1000024     %1.3e               # chargino1']

#Check if lines exist
with open(template,'r') as f:
    flines = f.readlines()
    for l in lines:
        if not any(l in lf for lf in flines):
            print('Line:\n %s \n not found in %s' %(l,template))

### Parameters to generate files

In [4]:
m1 = np.arange(100.,700.,50.)
width = np.logspace(-17,-13,30)
filename = 'TDTM1M2F_%1.0f_%1.1e_%1.0f_%1.0f_%1.1e_%1.0f.slha'
tempFolder = 'TDTM1M2F_slha'
tarball = 'TDTM1M2F.tar.gz'
if not os.path.isdir(tempFolder):
    os.mkdir(tempFolder)

In [5]:
with open(template,'r') as f:
    data = f.read()

files = []
for m in m1:    
    for w in width:
        
        #Define relations (constraints) between parameters
        m0 = m-0.5 #LSP mass
        
        #New file name
        fnew = os.path.join(tempFolder,filename %(m,w,m0,m,w,m0))
        
        #Define values to be replaced
        values = [m0,m,w] #Values should be sorted according to the ordering in lines
        
        #Replace lines
        dataNew = data[:]
        for l,newl,v in zip(lines,newlines,values):          
            dataNew = dataNew.replace(l, newl %v)

        #Write new file
        with open(fnew,'w') as ff:
            ff.write(dataNew+'\n')
            fullPath = os.path.abspath(fnew)
            #Add ref cross-sections
#            c = subprocess.run("./addRefXSecs.py -f %s -p 1000021 -q 1000021 -s 13" %fullPath,
#                                cwd="./slha",capture_output=True,shell=True)
            #or compute xsecs:
            c = subprocess.run("./smodelsTools.py xseccomputer -f %s -s 13 -8 -p" %fullPath,
                               cwd=os.path.expanduser('~/smodels'),capture_output=True,shell=True)
    
        files.append(fullPath)
print('%i files created' %len(files))

360 files created


### Create tarball and remove temp folder

In [6]:
with tarfile.open(tarball, "w:gz") as tar:
    tar.add(tempFolder, arcname='.')
# shutil.rmtree(tempFolder)    